In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
import multiprocessing
from collections import OrderedDict, defaultdict
from itertools import product
from copy import deepcopy, copy
from namegenerator import NameGenerator

TODO:
    - Add Bias term
    - Custom weights
    - Drop out connections (set weight to 0)
    - Custom activation per node
    
    - Add function that resets stagnation for all species\
    - Network Visualisation
    - Species Visualisation

# Activation functions

In [2]:
def leaky_relu(x):
    return F.leaky_relu(x)

def tanh(x):
    return F.Tanh(x)

def relu(x):
    return F.relu(x)

def sigmoid(x):
    return F.Sigmoid(x)

string_to_activation = {
    'leaky_relu' : leaky_relu,
    'relu' : relu,
    'sigmoid' : sigmoid,
    'tanh' : tanh
}

# Model

In [3]:
class Model(nn.Module):
    def __init__(self,layer_sizes):
        super(Model, self).__init__()
        layers = OrderedDict()
        
        previous_layer_size = layer_sizes[0]
        for idx, current_layer_size in enumerate(layer_sizes[1:]):
            layers[str(idx)] = nn.Linear(previous_layer_size, current_layer_size)
            previous_layer_size = current_layer_size
            
        self.layers = nn.Sequential(layers)
        
    def forward(self, x):
        return self.model(x)

In [4]:
firstname_generator = NameGenerator('names.csv', 3, 12)
new_individual_name = firstname_generator.generate_name()
previous_names = []
surname_generator = NameGenerator('surnames.csv', 3, 12)
new_specie_name = surname_generator.generate_name()
previous_surnames = []

# Genotype

In [5]:
class Genotype(object):
    def __init__(self, inputs = 144, 
                 outputs = 12, 
                 nonlinearities = ['relu','sigmoid','tanh'],
                 topology = None,
                 feedforward = True,
                 max_depth=None,
                 max_nodes = float('inf'),
                 initial_weight_stdev = 2.0,
                 bias_as_node = False,
                 p_add_neuron = 0.03, 
                 p_add_connection = 0.3, 
                 p_mutate_weight = 0.8,
                 p_reset_weight = 0.1,
                 p_reenable_connection = 0.01,
                 p_disable_connection = 0.01, 
                 p_reenable_parent = 0.25, 
                 p_mutate_bias = 0.2,
                 p_mutate_type = 0.2,
                 stdev_mutate_weight = 1.5,
                 stdev_mutate_bias = 0.5,
                 weight_range = (-50.,50.),
                 distance_excess_weight = 1.0, 
                 distance_disjoint_weight = 1.0, 
                 distance_weight = 0.4):
        
        self.name = next(new_individual_name)
        self.specie = None
        
        self.inputs = inputs
        self.outputs = outputs
        self.nonlinearities = nonlinearities
        self.feedforward = feedforward
        self.bias_as_node = bias_as_node
        
        self.max_depth = max_depth
        self.max_nodes = max_nodes
        
        self.initial_weight_stdev = initial_weight_stdev
        self.stdev_mutate_weight = stdev_mutate_weight
        self.stdev_mutate_bias = stdev_mutate_bias
        self.weight_range = weight_range
        
        # Mutation Probabilities
        self.p_add_neuron = p_add_neuron
        self.p_add_connection = p_add_connection
        self.p_mutate_weight = p_mutate_weight
        self.p_reset_weight = p_reset_weight
        self.p_reenable_connection = p_reenable_connection
        self.p_disable_connection = p_disable_connection
        self.p_reenable_parent = p_reenable_parent
        self.p_mutate_bias = p_mutate_bias
        self.p_mutate_type = p_mutate_type
        
        # Distance weights
        self.distance_excess_weight = distance_excess_weight
        self.distance_disjoint_weight = distance_disjoint_weight
        self.distance_weight = distance_weight
        
        # Tuples of: id, non_linearity, layer
        self.neuron_genes = []
        # Tuples of: innovation number, input, output, weight, enabled
        self.connection_genes = {}
        # Hyperparameter genes
        self.hyperparameter_genes = []
        
        self._initialise_topology(topology)
    
    def change_specie(self,specie):
        self.specie = specie
        
    def _initialise_topology(self, topology):
        if topology is None:
            # Initialise inputs
            for i in range(self.inputs):
                self.neuron_genes.append([i * 2048, random.choice(self.nonlinearities),1.0,0])

            # Initialise outputs
            for i in range(self.outputs):
                self.neuron_genes.append([(self.inputs + i) * 2048, random.choice(self.nonlinearities),1.0,0])

            # Initialise connections
            innovation_number = 0
            for i in range(self.inputs):
                for j in range(self.inputs,self.inputs + self.outputs):
                    weight = self._initialise_weight(self.inputs,self.outputs)
                    self.connection_genes[(i,j)] = [innovation_number, i, j, weight ,True]
                    innovation_number += 1
        else:
            raise NotImplementedError

                
    def _initialise_weight(self, input_neurons, output_neurons):
        weight = np.random.rand()*np.sqrt(1/(input_neurons + output_neurons))
        return weight
        
    def recombinate(self, other):
        child = deepcopy(self)
        child.neuron_genes = []
        child.connection_genes = {}
        
        max_neurons = max(len(self.neuron_genes), len(other.neuron_genes))
        min_neurons = min(len(self.neuron_genes), len(other.neuron_genes))
        
        for i in range(max_neurons):
            neuron_gene = None
            if i < min_neurons:
                neuron_gene = random.choice((self.neuron_genes[i], other.neuron_genes[i]))
            else:
                try:
                    neuron_gene = self.neuron_genes[i]
                except IndexError:
                    neuron_gene = other.neuron_genes[i]
            child.neuron_genes.append(deepcopy(neuron_gene))
            
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))
        max_innovation_number = max(list(self_connections.keys()) + list(other_connections.keys()))
        
        for i in range(max_innovation_number + 1):
            connection_gene = None
            if i in self_connections and i in other_connections:
                connection_gene = random.choice((self_connections[i],other_connections[i]))
                enabled = self_connections[i][4] and other_connections[i][4]
            else:
                if i in self_connections:
                    connection_gene = self_connections[i]
                    enabled = connection_gene[4]
                elif i in other_connections:
                    connection_gene = other_connections[i]
                    enabled = connection_gene[4]
            if connection_gene is not None:
                child.connection_genes[(connection_gene[1],connection_gene[2])] = deepcopy(connection_gene)
                child.connection_genes[(connection_gene[1],connection_gene[2])][4] = enabled or np.random.rand() < self.p_reenable_parent

            def is_feedforward(item):
                ((fr, to), cg) = item
                return child.neuron_genes[fr][0] < child.neuron_genes[to][0]

            if self.feedforward:
                child.connection_genes = dict(filter(is_feedforward, child.connection_genes.items()))
            
        return child
        
    def mutate(self, innovations = {}, global_innovation_number = 0):
        maximum_innovation_number = max(global_innovation_number, max(cg[0] for cg in self.connection_genes.values()))
        # TODO: move to separate functions
        if len(self.neuron_genes) < self.max_nodes and np.random.rand() < self.p_add_neuron:
            possible_to_split = self.connection_genes.keys()
            
            if self.max_depth is not None:
                possible_to_split = [(fr, to) for (fr, to) in possible_to_split if self.neuron_genes[fr][4] + 1 < self.neuron_genes[to][4]]
            
            if possible_to_split:

                # Choose connection to split
                split_neuron = self.connection_genes[random.choice(list(self.connection_genes.keys()))]
                # Disable old connection
                split_neuron[4] = False

                input_neuron, output_neuron, weight = split_neuron[1:4]
                neuron_id = (self.neuron_genes[input_neuron][0] + self.neuron_genes[input_neuron][0]) * 0.5
                nonlinearity = random.choice(self.nonlinearities)
                layer = self.neuron_genes[input_neuron][3] + 1

                neuron = [neuron_id, nonlinearity, 1.0, layer]

                new_id = len(self.neuron_genes)

                self.neuron_genes.append(neuron)
                
                if (input_neuron, new_id) in innovations:
                    innovation_number = innovations[(input_neuron,new_id)]
                else:
                    maximum_innovation_number += 1
                    innovation_number = innovations[(input_neuron,new_id)] = maximum_innovation_number
                    
                # 1.0 to initialise_weight?
                self.connection_genes[(input_neuron, new_id)] = [innovation_number, input_neuron, new_id, 1.0, True]
                
                if (new_id, output_neuron) in innovations:
                    innovation_number = innovations[(new_id, output_neuron)]
                else:
                    maximum_innovation_number += 1
                    innovation_number = innovations[(new_id, output_neuron)] = maximum_innovation_number
                    
                self.connection_genes[(new_id, output_neuron)] = [innovation_number, new_id, output_neuron, weight, True]

        elif np.random.rand() < self.p_add_connection:
            potential_connections = product(range(len(self.neuron_genes)),range(self.inputs, len(self.neuron_genes)))
            potential_connections = (connection for connection in potential_connections if connection not in self.connection_genes)
            
            if self.feedforward:
                potential_connections = ((f, t) for (f, t) in potential_connections if self.neuron_genes[f][0] < self.neuron_genes[t][0])
            
            potential_connections = list(potential_connections)
            if potential_connections:
                (fr, to) = random.choice(potential_connections)
                if (fr, to) in innovations:
                    innovation = innovations[(fr, to)]
                else:
                    maximum_innovation_number += 1
                    innovation = innovations[(fr, to)] = maximum_innovation_number
                # get number of neurons in layers of fr and to
                connection_gene = [innovation, fr, to, self._initialise_weight(2,2), True]
                self.connection_genes[(fr, to)] = connection_gene
        else:
            for cg in self.connection_genes.values():
                if np.random.rand() < self.p_mutate_weight:
                    cg[3] += np.random.normal(0, self.stdev_mutate_weight)
                    cg[3] = np.clip(cg[3], self.weight_range[0], self.weight_range[1])
                    # clipping?
                if np.random.rand() < self.p_reset_weight:
                    cg[3] = np.random.normal(0,self.stdev_mutate_weight)
                    
                # bigger chance to disable in this way
                if np.random.rand() < self.p_reenable_connection:
                    cg[4] = True
                    
                if np.random.rand() < self.p_disable_connection:
                    cg[4] = False
                    
            for neuron_gene in self.neuron_genes[self.inputs:]:
                if np.random.rand() < self.p_mutate_bias:
                    neuron_gene[2] += np.random.normal(0, 1)
                    neuron_gene[2] = np.clip(neuron_gene[2], self.weight_range[0], self.weight_range[1])
                
                if np.random.rand() < self.p_mutate_type:
                    neuron_gene[1] = random.choice(self.nonlinearities)   
                    
        return self
        
    def distance(self, other):
        self_connections = dict(((c[0], c) for c in self.connection_genes.values()))
        other_connections = dict(((c[0], c) for c in other.connection_genes.values()))

        all_innovations = list(self_connections.keys()) + list(other_connections.keys())

        minimum_innovation = min(all_innovations)
        
        e = 0
        d = 0
        w = 0.0
        m = 0
        
        for innovation_key in all_innovations:
            if innovation_key in self_connections and innovation_key in other_connections:
                w += np.abs(self_connections[innovation_key][3] - other_connections[innovation_key][3])
                m += 1
            elif innovation_key in self_connections or innovation_key in other_connections:
                # Disjoint genes
                if innovation_key < minimum_innovation:
                    d += 1
                # Excess genes
                else:
                    e += 1
                    
        # Average weight differences of matching genes
        w = (w/m) if m>0 else w
        
        return (self.distance_excess_weight * e +
               self.distance_disjoint_weight * d +
               self.distance_weight * w)
    
    def get_network_data(self):
        """ Returns a tuple of (connection_matrix, node_types) 
            that is reordered by the "feed-forward order" of the network,
            Such that if feedforward was set to true, the matrix will be
            lower-triangular.
            The node bias is inserted as "node 0", the leftmost column
            of the matrix.
        """
        
        # Assemble connectivity matrix
        cm = np.zeros((len(self.neuron_genes), len(self.neuron_genes)))
        cm.fill(np.nan)
        
        for (_, fr, to, weight, enabled) in self.connection_genes.values():
            if enabled:
                cm[to, fr] = weight
        
        # Reorder the nodes/connections
#         ff, node_types, bias, response, layer = zip(*self.neuron_genes)
        ff, node_types, bias, layer = zip(*self.neuron_genes)
        order = [i for _,i in sorted(zip(ff, range(len(ff))))]
        cm = cm[:,order][order,:]
        node_types = np.array(node_types)[order]
        bias = np.array(bias)[order]
#         response = np.array(response)[order]
        layers = np.array(layer)[order]

#         # Then, we multiply all the incoming connection weights by the response
        cm *= np.atleast_2d(4.9).T
        # Finally, add the bias as incoming weights from node-0
        if not self.bias_as_node:
            cm = np.hstack( (np.atleast_2d(bias).T, cm) )
            cm = np.insert(cm, 0, 0.0, axis=0)
            # TODO: this is a bit ugly, we duplicate the first node type for 
            # bias node. It shouldn't matter though since the bias is used as an input.
            node_types = [node_types[0]] + list(node_types)

        if self.feedforward and np.triu(np.nan_to_num(cm)).any():
            import pprint
            pprint.pprint(self.neuron_genes)
            pprint.pprint(self.connection_genes)
            print(ff)
            print(order)
            print(np.sign(cm))
            raise Exception("Network is not feedforward.")
        
        return cm, node_types

# Species

In [6]:
class Species(object):
    def __init__(self, initial_member):
        self.name = next(new_specie_name)
        self.members = [initial_member]
        self.representative = initial_member
        self.offspring = 0
        self.age = 0
        self.average_fitness = 0.
        self.max_fitness = 0.
        self.max_fitness_previous = 0.0
        self.stagnation = 0
        self.has_best = False

# Population

In [7]:
class Population(object):
    def __init__(self, genome_factory,
                population_size = 100,
                elitism = True,
                stop_when_solved = False,
                tournament_selection_k = 3,
                verbose = True,
                max_cores = 1,
                compatibility_threshold = 3.0,
                compatibility_threshold_delta = 0.4,
                target_species = 12,
                minimum_elitism_size = 5,
                young_age = 10,
                young_multiplier = 1.2,
                old_age = 30,
                old_multiplier = 0.2,
                stagnation_age = 15,
                reset_innovations = False,
                survival = 0.2):
        
        self.genome_factory = genome_factory
        self.population_size = population_size
        self.elitism = elitism
        self.stop_when_solved = stop_when_solved
        self.tournament_selection_k = tournament_selection_k
        self.verbose = verbose
        self.max_cores = max_cores
        
        cpus = multiprocessing.cpu_count()
        use_cores = min(self.max_cores, cpus-1)
        if use_cores > 1:
            self.pool = multiprocessing.Pool(processes=use_cores, maxtasksperchild=5)
        else:
            self.pool = None
        
        self.compatibility_threshold = compatibility_threshold
        self.compatibility_threshold_delta = compatibility_threshold_delta
        
        self.target_species = target_species
        self.minimum_elitism_size = minimum_elitism_size
        
        self.young_age = young_age
        self.young_multiplier = young_multiplier
        self.old_age = old_age
        self.old_multiplier = old_multiplier
        
        self.stagnation_age = stagnation_age
        
        self.reset_innovations = reset_innovations
        self.survival = survival
    
    # Peas has this function outside the class
    def evaluate_individual(self,item):
        (individual, evaluator) = item
        if callable(evaluator):
            individual.stats = evaluator(individual)
        elif hasattr(evaluator, 'evaluate'):
            individual.stats = evaluator.evaluate(individual)
        else:
            raise Exception("Evaluator must be a callable or object" \
                        "with a callable attribute 'evaluate'.")
        return individual
        
    def _evaluate_all(self, population, evaluator):
        to_eval = [(individual, evaluator) for individual in population]
        if self.pool is not None:
            population = list(self.pool.map(self.evaluate_indivual, to_eval))
        else:
            population = list(map(self.evaluate_individual, to_eval))
        
        return population
        
    def _reset(self):
        self.champions = []
        self.generation = 0
        self.solved_at = None
        self.stats = defaultdict(list)
        self.species = []
        self.global_innovation_number = 0
        self.innovations = {}
        self.current_compatibility_threshold = self.compatibility_threshold
        
    def _find_best(self, population, solution = None):
        self.champions.append(max(population, key=lambda individual: individual.stats['fitness']))
        
        if solution is not None:
            if isinstance(solution, (int, float)):
                solved = (self.champions[-1].stats['fitness'] >= solution)
            elif callable(solution):
                solved = solution(self.champions[-1])
            elif hasattr(solution, 'solve'):
                solved = solution.solve(self.champions[-1])
                
            if solved and self.solved_at is None:
                self.solved_at = self.generation + 1
            
    @property
    def population(self):
        for specie in self.species:
            for member in specie.members:
                yield member
    
    def _evolve(self, evaluator, solution=None):
        population = list(self.population)
        
        while len(population) < self.population_size:
            individual = self.genome_factory()
            population.append(individual)        
            
        population = self._evaluate_all(population, evaluator)
        
        # Speciation
        for specie in self.species:
            # Choose random specie representative for distance comparison
            specie.representative = random.choice(specie.members)
            specie.name = specie.representative.specie
            specie.members = []
            specie.age += 1
            
        # Add each individual to a species
        for individual in population:
            found = False
            for specie in self.species:
                if individual.distance(specie.representative) <= self.current_compatibility_threshold:
                    specie.members.append(individual)
                    individual.change_specie(specie.name)
                    found = True
                    break
            if not found:
                s = Species(individual)
                individual.change_specie(s.name)
                self.species.append(s)
        
        # Remove empty species
        self.species = list(filter(lambda s: len(s.members) > 0, self.species))
        
        # Adjust compatibility threshold
        if len(self.species) < self.target_species:
            self.current_compatibility_threshold -= self.compatibility_threshold_delta
        elif len(self.species) > self.target_species:
            self.current_compatibility_threshold += self.compatibility_threshold_delta
        
        # Find champion and check for solution
        self._find_best(population, solution)
        
        # Recombination
        
        for specie in self.species:
            specie.max_fitness_previous = specie.max_fitness
            specie.average_fitness = np.mean([individual.stats['fitness'] for individual in specie.members])
            specie.max_fitness = np.max([individual.stats['fitness'] for individual in specie.members])
            if specie.max_fitness <= specie.max_fitness_previous:
                specie.stagnation += 1
            else:
                specie.stagnation = 0
            specie.has_best = self.champions[-1] in specie.members
        
        # Keep species that have the best or within stagnation age range
        self.species = list(filter(lambda s: s.stagnation < self.stagnation_age or s.has_best, self.species))
        
        average_fitness = np.array([specie.average_fitness for specie in self.species])
        
        # Adjust fitness based on age
        age = np.array([specie.age for specie in self.species])
        for specie in self.species:
            if specie.age < self.young_age:
                specie.average_fitness *= self.young_multiplier
            if specie.age > self.old_age:
                specie.average_fitness *= self.old_multiplier
                
        # Compute offspring size
        total_fitness = sum(specie.average_fitness for specie in self.species)
        for specie in self.species:
            specie.offspring = int(round(self.population_size * specie.average_fitness / total_fitness))
            
        
        
        # Remove species without offspring
        self.species = list(filter(lambda s: s.offspring > 0, self.species))

        for specie in self.species:
            specie.members.sort(key=lambda individual: individual.stats['fitness'], reverse = True)
            keep = max(1, int(round(len(specie.members)*self.survival)))
            pool = specie.members[:keep]
            
            if self.elitism and len(specie.members) > self.minimum_elitism_size:
                specie.members = specie.members[:1]
            else:
                specie.members = []
                
            while len(specie.members) < specie.offspring:
                k = min(len(pool), self.tournament_selection_k)
                p1 = max(random.sample(pool,k), key=lambda individual: individual.stats['fitness'])
                p2 = max(random.sample(pool,k), key=lambda individual: individual.stats['fitness'])
                
                child = p1.recombinate(p2)
                child.mutate(innovations=self.innovations, global_innovation_number = self.global_innovation_number)
                specie.members.append(child)
                
        if self.innovations:
            self.global_innovation_number = max(self.innovations.values())
            
        self._gather_stats(population)
            
    def epoch(self, evaluator, generations, solution=None, reset=True, callback= None):
        if reset:
            self._reset()
            
        for i in range(generations):
            self._evolve(evaluator, solution)
            self.generation += 1
            
            if self.verbose:
                self._status_report()
                
            if callback is not None:
                callback(self)
            
            if self.solved_at is not None and self.stop_when_solved:
                break
        
        return {'stats': self.stats, 'champions': self.champions}
    
    def _gather_stats(self, population):
        for key in population[0].stats:
            self.stats[key+'_avg'].append(np.mean([individual.stats[key] for individual in population]))
            self.stats[key+'_max'].append(np.max([individual.stats[key] for individual in population]))
            self.stats[key+'_min'].append(np.min([individual.stats[key] for individual in population]))
        self.stats['solved'].append( self.solved_at is not None )
    
    def _status_report(self):
        print("\n****** Running Generation %d ******" % self.generation)
        fitness_list = np.array([i.stats['fitness'] for i in self.population])
        number_neurons = len(self.champions[-1].neuron_genes)
        number_enabled_connections = np.sum([1 for conn in self.champions[-1].connection_genes.values() if conn[4]])
        print("Population's average fitness: %.5f stdev: %.5f" % (np.average(fitness_list), np.std(fitness_list)))
        print("Best individual: %s %s" % (self.champions[-1].specie, self.champions[-1].name))
        print("Best fitness: %.2f - #neurons: %i - #enabled connections: %i" % (self.champions[-1].stats['fitness'],number_neurons,number_enabled_connections))
        print("Population of %i members in %i species:" % (len(list(self.population)), len(self.species)))
        print("Species         age    size    fitness    stag")
        print("============    ===    ====    =======    ====")
        for specie in self.species:
            print("{: >12}    {: >3}    {: >4}    {:.5f}    {: >4}".format(specie.name,specie.age,len(specie.members),specie.max_fitness,specie.stagnation))
        print("Solved: %s" % (self.solved_at))

# Test

In [8]:
""" Package with some classes to simulate neural nets.
"""

### IMPORTS ###

import sys
import numpy as np
np.seterr(over='ignore', divide='raise')

# Libraries

# Local


# Shortcuts

inf = float('inf')
sqrt_two_pi = np.sqrt(np.pi * 2)

### FUNCTIONS ###

# Node functions
def ident(x):
    return x

def bound(x, clip=(-1.0, 1.0)):
    return np.clip(x, *clip)

def gauss(x):
    """ Returns the pdf of a gaussian.
    """
    return np.exp(-x ** 2 / 2.0) / sqrt_two_pi
    
def sigmoid(x):
    """ Sigmoid function. 
    """
    return 1 / (1 + np.exp(-x))

def sigmoid2(x):
    """ Sigmoid function. 
    """
    return 1 / (1 + np.exp(-4.9*x))

def abs(x):
    return np.abs(x)

def sin(x):
    return np.sin(x)

def tanh(x):
    return np.tanh(x)

def summed(fn):
    return lambda x: fn(sum(x))

def relu(x):
    return x * (x>0)

### CONSTANTS ###

SIMPLE_NODE_FUNCS = {
    'sin': np.sin,
    'abs': np.abs,
    'ident': ident,
    'linear': ident,
    'bound': bound,
    'gauss': gauss,
    'sigmoid': sigmoid,
    'sigmoid2': sigmoid2,
    'exp': sigmoid,
    'tanh': tanh,
    'relu': relu,
    None : ident
}

def rbfgauss(x):
    return np.exp(-(x ** 2).sum() / 2.0) / sqrt_two_pi

def rbfwavelet(x):
    return np.exp(-(x ** 2).sum() / ( 2* 0.5**2 )) * np.sin(2 * np.pi * x[0])

COMPLEX_NODE_FUNCS = {
    'rbfgauss': rbfgauss,
    'rbfwavelet': rbfwavelet
}



### CLASSES ### 

class NeuralNetwork(object):
    """ A neural network. Can have recursive connections.
    """
    
    def from_matrix(self, matrix, node_types=['sigmoid']):
        """ Constructs a network from a weight matrix. 
        """
        # Initialize net
        self.original_shape = matrix.shape[:matrix.ndim//2]
        # If the connectivity matrix is given as a hypercube, squash it down to 2D
        n_nodes = np.prod(self.original_shape)
        self.cm  = matrix.reshape((n_nodes,n_nodes))
        self.node_types = node_types
        if len(self.node_types) == 1:
            self.node_types *= n_nodes
        self.act = np.zeros(self.cm.shape[0])
        self.optimize()
        return self
        
    def from_neatchromosome(self, chromosome):
        """ Construct a network from a Chromosome instance, from
            the neat-python package. This is a connection-list
            representation.
        """
        # TODO Deprecate the neat-python compatibility
        # Typecheck
        import neat.chromosome
        
        if not isinstance(chromosome, neat.chromosome.Chromosome):
            raise Exception("Input should be a NEAT chromosome, is %r." % (chromosome))
        # Sort nodes: BIAS, INPUT, HIDDEN, OUTPUT, with HIDDEN sorted by feed-forward.
        nodes = dict((n.id, n) for n in chromosome.neuron_genes)
        node_order = ['bias']
        node_order += [n.id for n in filter(lambda n: n.type == 'INPUT', nodes.values())]
        if isinstance(chromosome, neat.chromosome.FFChromosome):
            node_order += chromosome.node_order
        else:
            node_order += [n.id for n in filter(lambda n: n.type == 'HIDDEN', nodes.values())]
        node_order += [n.id for n in filter(lambda n: n.type == 'OUTPUT', nodes.values())]
        # Construct object
        self.cm = np.zeros((len(node_order), len(node_order)))
        # Add bias connections
        for id, node in nodes.items():
            self.cm[node_order.index(id), 0] = node.bias
            self.cm[node_order.index(id), 1:] = node.response
        # Add the connections
        for conn in chromosome.connection_genes:
            if conn.enabled:
                to = node_order.index(conn.outnodeid)
                fr = node_order.index(conn.innodeid)
                # dir(conn.weight)
                self.cm[to, fr] *= conn.weight
        # Verify actual feed forward
        if isinstance(chromosome, neat.chromosome.FFChromosome):
            if np.triu(self.cm).any():
                raise Exception("NEAT Chromosome does not describe feedforward network.")
        node_order.remove('bias')
        self.node_types = [nodes[i].activation_type for i in node_order]
        self.node_types = ['ident'] + self.node_types
        self.act = np.zeros(self.cm.shape[0])
        self.optimize()
        return self

    def optimize(self):
        # If all nodes are simple nodes
        if all(fn in SIMPLE_NODE_FUNCS for fn in self.node_types):
            # Simply always sum the node inputs, this is faster
            self.sum_all_node_inputs = True
            self.cm = np.nan_to_num(self.cm)
            # If all nodes are identical types
            if all(fn == self.node_types[0] for fn in self.node_types):
                self.all_nodes_same_function = True
            self.node_types = [SIMPLE_NODE_FUNCS[fn] for fn in self.node_types]
        else:
            nt = []
            for fn in self.node_types:
                if fn in SIMPLE_NODE_FUNCS:
                    # Substitute the function(x) for function(sum(x))
                    nt.append(summed(SIMPLE_NODE_FUNCS[fn]))
                else:
                    nt.append(COMPLEX_NODE_FUNCS[fn])
            self.node_types = nt

    
    def __init__(self, source=None):
        # Set instance vars
        self.feedforward    = False
        self.sandwich       = False   
        self.cm             = None
        self.node_types     = None
        self.original_shape = None
        self.sum_all_node_inputs = False
        self.all_nodes_same_function = False
        
        if source is not None:
            try:
                self.from_matrix(*source.get_network_data())
                if hasattr(source, 'feedforward') and source.feedforward:
                    self.make_feedforward()
            except AttributeError as e:
                print(e)
                raise Exception("Cannot convert from %s to %s" % (source.__class__, self.__class__))

    def make_sandwich(self):
        """ Turns the network into a sandwich network,
            a network with no hidden nodes and 2 layers.
        """
        self.sandwich = True
        self.cm = np.hstack((self.cm, np.zeros(self.cm.shape)))
        self.cm = np.vstack((np.zeros(self.cm.shape), self.cm))
        self.act = np.zeros(self.cm.shape[0])
        return self
        
    def num_nodes(self):
        return self.cm.shape[0]
        
    def make_feedforward(self):
        """ Zeros out all recursive connections. 
        """
        if np.triu(np.nan_to_num(self.cm)).any():
            raise Exception("Connection Matrix does not describe feedforward network. \n %s" % np.sign(self.cm))
        self.feedforward = True
        self.cm[np.triu_indices(self.cm.shape[0])] = 0
        
    def flush(self):
        """ Reset activation values. """
        self.act = np.zeros(self.cm.shape[0])
        
    def feed(self, input_activation, add_bias=True, propagate=1):
        """ Feed an input to the network, returns the entire
            activation state, you need to extract the output nodes
            manually.
            
            :param add_bias: Add a bias input automatically, before other inputs.
        """
        if propagate != 1 and (self.feedforward or self.sandwich):
            raise Exception("Feedforward and sandwich network have a fixed number of propagation steps.")
        act = self.act
        node_types = self.node_types
        cm = self.cm
        input_shape = input_activation.shape
        
        if add_bias:
            input_activation = np.hstack((1.0, input_activation))
        
        if input_activation.size >= act.size:
            raise Exception("More input values (%s) than nodes (%s)." % (input_activation.shape, act.shape))
        
        input_size = min(act.size - 1, input_activation.size)
        node_count = act.size
        
        # Feed forward nets reset the activation, and activate as many
        # times as there are nodes
        if self.feedforward:
            act = np.zeros(cm.shape[0])
            propagate = len(node_types)
        # Sandwich networks only need to activate a single time
        if self.sandwich:
            propagate = 1
        for _ in range(propagate):
            act[:input_size] = input_activation.flat[:input_size]
            
            if self.sum_all_node_inputs:
                nodeinputs = np.dot(self.cm, act)
            else:
                nodeinputs = self.cm * act
                nodeinputs = [ni[-np.isnan(ni)] for ni in nodeinputs]
            
            if self.all_nodes_same_function:
                act = node_types[0](nodeinputs)
            else:
                for i in range(len(node_types)):
                    act[i] = node_types[i](nodeinputs[i])

        self.act = act

        # Reshape the output to 2D if it was 2D
        if self.sandwich:
            return act[act.size//2:].reshape(input_shape)      
        else:
            return act.reshape(self.original_shape)

    def cm_string(self):
        print("Connectivity matrix: %s" % (self.cm.shape,))
        cp = self.cm.copy()
        s = np.empty(cp.shape, dtype='a1')
        s[cp == 0] = ' '
        s[cp > 0] = '+'
        s[cp < 0] = '-'
        return '\n'.join([''.join(l) + '|' for l in s])

    
    def visualize(self, filename, inputs=3, outputs=1):
        """ Visualize the network, stores in file. """
        if self.cm.shape[0] > 50:
            return
        import pygraphviz as pgv
        # Some settings
        node_dist = 1
        cm = self.cm.copy()
        # Sandwich network have half input nodes.
        if self.sandwich:
            inputs = cm.shape[0] // 2
            outputs = inputs
        # Clear connections to input nodes, these arent used anyway

        G = pgv.AGraph(directed=True)
        mw = abs(cm).max()
        for i in range(cm.shape[0]):
            G.add_node(i)
            t = self.node_types[i].__name__
            G.get_node(i).attr['label'] = '%d:%s' % (i, t[:3])
            for j in range(cm.shape[1]):
                w = cm[i,j]
                if abs(w) > 0.01:
                    G.add_edge(j, i, penwidth=abs(w)/mw*4, color='blue' if w > 0 else 'red')
        for n in range(inputs):
            pos = (node_dist*n, 0)
            G.get_node(n).attr['pos'] = '%s,%s!' % pos
            G.get_node(n).attr['shape'] = 'doublecircle'
            G.get_node(n).attr['fillcolor'] = 'steelblue'
            G.get_node(n).attr['style'] = 'filled'
        for i,n in enumerate(range(cm.shape[0] - outputs,cm.shape[0])):
            pos = (node_dist*i, -node_dist * 5)
            G.get_node(n).attr['pos'] = '%s,%s!' % pos
            G.get_node(n).attr['shape'] = 'doublecircle'
            G.get_node(n).attr['fillcolor'] = 'tan'
            G.get_node(n).attr['style'] = 'filled'
        
        G.node_attr['shape'] = 'circle'
        if self.sandwich: 
            # neato supports fixed node positions, so it's better for
            # sandwich networks
            prog = 'neato'
        else:
            prog = 'dot'
        G.draw(filename, prog=prog)
        
    def __str__(self):
        return 'Neuralnet with %d nodes.' % (self.act.shape[0])
    

""" Input/output relation task. Every input and output
    is explicitly defined. XOR is an example of this task.
"""

### IMPORTS ###
import random

# Libraries
import numpy as np

# Local



class XORTask(object):
    
    # Default XOR input/output pairs
    INPUTS  = [(0,0), (0,1), (1,0), (1,1)]
    OUTPUTS = [(-1,), (1,), (1,), (-1,)]
    EPSILON = 1e-100
    
    def __init__(self, do_all=True):
        self.do_all = do_all
        self.INPUTS = np.array(self.INPUTS, dtype=float)
        self.OUTPUTS = np.array(self.OUTPUTS, dtype=float)
    
    def evaluate(self, network, verbose=False):
        if not isinstance(network, NeuralNetwork):
            network = NeuralNetwork(network)
        
        network.make_feedforward()
        
        pairs = list(zip(self.INPUTS, self.OUTPUTS))
        random.shuffle(pairs)
        if not self.do_all:
            pairs = [random.choice(pairs)]
        rmse = 0.0
        for (i, target) in pairs:
            # Feed with bias
            output = network.feed(i)
            # Grab the output
            output = output[-len(target):]
            err = (target - output)
            err[abs(err) < self.EPSILON] = 0;
            err = (err ** 2).mean()
            # Add error
            if verbose:
                print("%r -> %r (%.2f)" % (i, output, err))
            rmse += err 

        score = 1/(1+np.sqrt(rmse / len(pairs)))
        return {'fitness': score}
        
    def solve(self, network):
        return int(self.evaluate(network)['fitness'] > 0.9)

In [9]:
inputs = 3
outputs = 4
nonlinearities = ['tanh']
topology = None
feedforward = True
max_depth = None
max_nodes = float('inf')
bias_as_node = False
initial_weight_stdev = 2.0
p_add_neuron = 0.03
p_add_connection = 0.3
p_mutate_weight = 0.8
p_reset_weight = 0.1
p_reenable_connection = 0.01
p_disable_connection = 0.01
p_reenable_parent=0.25
p_mutate_bias = 0.2
p_mutate_type = 0.2
stdev_mutate_weight = 1.5
stdev_mutate_bias = 0.5
weight_range = (-50.,50.)

distance_excess_weight = 1.0
distance_disjoint_weight = 1.0
distance_weight = 0.4

In [10]:
population_size = 100
elitism = False
stop_when_solved = True 
tournament_selection_k = 3 
verbose = True
max_cores = 1
compatibility_threshold = 3.0
compatibility_threshold_delta = 0.4 
target_species = 12
minimum_elitism_size = 5 
young_age = 10
young_multiplier = 1.2 
old_age = 30
old_multiplier = 0.2 
stagnation_age = 15
reset_innovations = False
survival = 0.2

genome_factory = lambda: Genotype(inputs, outputs, nonlinearities, topology, feedforward, max_depth, max_nodes, initial_weight_stdev, bias_as_node, p_add_neuron, p_add_connection, p_mutate_weight, p_reset_weight, p_reenable_connection, p_disable_connection, p_reenable_parent, p_mutate_bias, p_mutate_type, stdev_mutate_weight, stdev_mutate_bias, weight_range, distance_excess_weight, distance_disjoint_weight, distance_weight)

population = Population(genome_factory, population_size, elitism, stop_when_solved, tournament_selection_k, verbose, max_cores, compatibility_threshold, compatibility_threshold_delta, target_species, minimum_elitism_size, young_age, young_multiplier, old_age, old_multiplier, stagnation_age, reset_innovations, survival)
task = XORTask()

result = population.epoch(evaluator = task, generations = 1000, solution = task)


****** Running Generation 1 ******
Population's average fitness: 0.42802 stdev: 0.00267
Best individual: sal oma
Best fitness: 0.43 - #neurons: 7 - #enabled connections: 12
Population of 100 members in 1 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
         sal      0     100    0.43289       0
Solved: None

****** Running Generation 2 ******
Population's average fitness: 0.46581 stdev: 0.02793
Best individual: sal urindan
Best fitness: 0.50 - #neurons: 7 - #enabled connections: 12
Population of 100 members in 1 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
         sal      1     100    0.49990       0
Solved: None

****** Running Generation 3 ******
Population's average fitness: 0.46542 stdev: 0.02239
Best individual: sal urindan
Best fitness: 0.50 - #neurons: 7 - #enabled connections: 13
Population of 100 members in 4 species:
Species         age    size    fitness    


****** Running Generation 11 ******
Population's average fitness: 0.49187 stdev: 0.02339
Best individual: sal reedane
Best fitness: 0.50 - #neurons: 7 - #enabled connections: 13
Population of 99 members in 11 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
         sal     10       8    0.50000       2
      bronni      8       9    0.50000       0
          ri      7       9    0.50000       1
brianicekars      7       9    0.48733       3
          he      6       9    0.49977       1
         cle      6       9    0.50000       0
          so      6       9    0.50000       1
       anton      5       8    0.41392       1
         krs      5      10    0.50000       0
       popov      4       9    0.50000       0
        tard      4      10    0.50000       0
Solved: None

****** Running Generation 12 ******
Population's average fitness: 0.49098 stdev: 0.02212
Best individual: sal reedane
Best fitness: 0.50 - #neurons: 7 - #en


****** Running Generation 21 ******
Population's average fitness: 0.49984 stdev: 0.00058
Best individual: sal reedane
Best fitness: 0.50 - #neurons: 7 - #enabled connections: 13
Population of 100 members in 11 species:
Species         age    size    fitness    stag
============    ===    ====    =======    ====
         sal     20       9    0.50000      12
      bronni     18       8    0.50000       6
          ri     17       9    0.50000       3
         cle     16       8    0.50000       8
         krs     15       8    0.50000       7
       popov     14       7    0.49774       0
         bin      8       9    0.50000       0
      heimen      8      10    0.50000       0
       bield      2      10    0.50000       2
        latz      0      11    0.50000       0
          au      0      11    0.50000       0
Solved: None

****** Running Generation 22 ******
Population's average fitness: 0.48628 stdev: 0.03001
Best individual: sal reedane
Best fitness: 0.50 - #neurons: 8 - #e

In [11]:
w = torch.empty(3, 5)
nn.init.xavier_uniform_(w, gain=nn.init.calculate_gain('relu'))

tensor([[-0.1557,  0.6790,  1.2225, -0.5605, -0.0323],
        [-0.6397, -0.1951,  0.3126,  0.9808, -0.4102],
        [-0.8842,  1.1706, -0.5982,  0.6167,  0.3099]])